In [57]:
import pandas as pd
import numpy as np
from scipy.stats import chisquare
import xgboost as xgb
ch = pd.read_csv('for_chisq_full.csv')
print(ch.shape)

(5443, 4)


In [64]:
ch['obs'] = ch['cnt']*ch['mean']
ch['exp'] = ch['cnt']*0.3351935

In [71]:
stats = []
for i in range(len(ch)):
    st = chisquare(ch['obs'][i], ch['exp'][i])
    stats.append(st.statistic)

In [73]:
ch['stats'] = stats

In [78]:
ch.sort_values(axis = 0, by = 'stats', ascending=False).to_csv('ch_result.csv', index=None)

In [58]:

chisquare([3558*0.133, 3558*0.3351935])

Power_divergenceResult(statistic=310.6811784262821, pvalue=1.55179759439844e-69)

In [63]:
chisquare(ch['cnt'][:5]*ch['mean'][:5])

Power_divergenceResult(statistic=1275.93656457528, pvalue=5.4875017778639486e-275)

In [2]:
dt = pd.read_parquet('train.parquet')

In [3]:
features_base = ['dest_number count', 'voice_ratio mean', 'freq_msg mean',
       'lifetime max', 'lifetime min', 'cnt_voice_out sum', 'cnt_msg_out sum',
       'freq_voice mean', 'cnt_msg_in sum', 'cnt_voice_in sum',
       'msg_ratio mean', 'last_contact min', 'last_contact mean']

freq = [
'OKru','DpyrBokpyr','Google','4446','MMS','4449','5557','QIWIWallet','MailRu','WhatsApp','WEBBANKIR',
'Bistrodengi','Youla','OTTService','Access','159','Avito','SBERBANK','545','78007072279','ekapusta',
'SMSfinance','MSG','Yandex','sangistil','78005050400','mili.ru','597','15400','73422700006','LICARD',
'Zaymer.ru','CLUB-LUKOIL','78001000800','moneyman','tabor.ru','gogomusic.net','Denga','VashiDengi',
'VIVADENGI','centrofin','SMS','MigCredit','aptOtSklada','ICQ','Viber','Chitaensbit','74957923097',
'74957923096','Medlab.','OTKRITIE','pecom.ru','74992905223','r-ulybka.ru','Partner','RGS','Facebook','Aeroflot'
]

features = features_base + freq

dm = xgb.DMatrix(data = dt[features].values
                 , label = dt['label'].values
                 , feature_names=features
                 , missing = np.nan)

params = {'objective':'binary:logistic',
         'eval_metric':'auc',
          'max_depth':4, #maximum depth of tree
          'eta':0.1, #learning rate
          'gamma':5, # regularization 1
          'seed':321, # reproducibility
         'subsample':0.7, #subsampling 1
         'colsample_bytree':0.7, #subsampling 2
         'min_child_weight':5, # regularization 2
         'scale_pos_weight':1, ## Normally this is an integer > 1
         'silent':False}

nrounds = 200
xgb.cv(params, dm, nrounds)

,test-auc-mean,test-auc-std,train-auc-mean,train-auc-std
0,0.722168,0.009877,0.722942,0.010922
1,0.742691,0.002764,0.743483,0.003960
2,0.747496,0.001489,0.748560,0.001070
3,0.750035,0.001466,0.751086,0.001914
4,0.751076,0.000950,0.752194,0.000532
5,0.752309,0.000683,0.753593,0.000573
6,0.753421,0.001340,0.754780,0.000072
7,0.754026,0.001215,0.755377,0.000084
8,0.754385,0.001276,0.755767,0.000135
9,0.755252,0.001475,0.756562,0.000264


In [4]:
booster = xgb.train(params, dm, nrounds)
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('whitegrid')
xgb.plot_importance(booster)
plt.show()

<Figure size 640x480 with 1 Axes>

In [8]:
booster.save_model('rbt_xgb.xgb')

In [9]:
dt['pred'] = booster.predict(dm)

In [10]:
dt.head()

,dest_number count,voice_ratio mean,freq_msg mean,lifetime max,lifetime min,cnt_voice_out sum,cnt_msg_out sum,freq_voice mean,cnt_msg_in sum,cnt_voice_in sum,...,OTKRITIE,pecom.ru,74992905223,r-ulybka.ru,Partner,RGS,Facebook,Aeroflot,label,pred
subs_id,,,,,,,,,,,,,,,,,,,,,
162345,574.0,736.0,793.0,12483.0,525,0.026568,0.047741,inf,0.105485,790.0,...,279.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0.515438
168521,1357.0,1037.0,8.0,1198.0,95,0.046728,0.071036,inf,0.020214,790.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0.228036
171056,11800.0,6975.0,223.0,5852.0,2619,0.021631,0.010660,inf,inf,790.0,...,NaN,272.0,NaN,NaN,NaN,NaN,NaN,NaN,0,0.175024
172972,573.0,427.0,2.0,1638.0,91,0.019908,0.026511,inf,0.000154,790.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0.084944
188327,3734.0,4372.0,218.0,6102.0,1210,0.019498,0.016156,inf,inf,790.0,...,NaN,NaN,NaN,NaN,NaN,NaN,748.0,NaN,1,0.555748
